# lib

In [5]:
import sentencepiece as spm
import torch
import torch.nn as nn

# vocab

In [3]:
vocab_file = "news2.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

# 입력 texts
lines = [
  "겨울은 추워요.",
  "감기 조심하세요."
]

# text를 tensor로 변환(텍스트 데이터를 숫자로 allow computer to understand text)
inputs = []
for line in lines:
  pieces = vocab.encode_as_pieces(line)
  ids = vocab.encode_as_ids(line)
  inputs.append(torch.tensor(ids))
  print(pieces)

# 입력 길이가 다르므로 입력 최대 길이에 맟춰 padding(0)을 추가 해 줌
inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
# shape
print(inputs.size())
# 값
print(inputs)

['▁', '겨울은', '▁', '추워요', '.']
['▁', '감기', '▁', '조심하세요', '.']
torch.Size([2, 5])
tensor([[7921,    1, 7921,    1, 7942],
        [7921,    1, 7921,    1, 7942]])


# embedding
transformer uses input embedding and position embedding

## input embedding
입력 텍스트(단어, 문장 등)를 모델이 이해할 수 있는 숫자 벡터로 변환하는 과정

In [6]:
n_vocab = len(vocab) # vocab count
d_hidn = 128 # hidden size
nn_emb = nn.Embedding(n_vocab, d_hidn) # embedding 객체

input_embs = nn_emb(inputs) # input embedding
print(input_embs.size())

torch.Size([2, 5, 128])


## position embedding

- 각 position별도 angle 값을 구합니다.
- 구해진 angle 중 짝수 index의 값에 대한 sin 값을 구합니다.
- 구해진 angle 중 홀수 index의 값에 대한 cos 값을 구합니다.

In [7]:
""" sinusoid position embedding """
def get_sinusoid_encoding_table(n_seq, d_hidn):
    def cal_angle(position, i_hidn):
        return position / np.power(10000, 2 * (i_hidn // 2) / d_hidn)
    def get_posi_angle_vec(position):
        return [cal_angle(position, i_hidn) for i_hidn in range(d_hidn)]

    sinusoid_table = np.array([get_posi_angle_vec(i_seq) for i_seq in range(n_seq)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # even index sin 
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # odd index cos

    return sinusoid_table